# Part of Speech (POS) tagging with Hidden Markov Model (HMM)

In [25]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

In [26]:

#download the treebank corpus from nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\panka\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [27]:
#download the universal tagset from nltk
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\panka\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [28]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [29]:
#print the first two sentences along with tags
print(nltk_data[:2])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]]


In [30]:
#print each word with its respective tag for first two sentences
for sent in nltk_data[:2]:
  for tuple in sent:
    print(tuple)

('Pierre', 'NOUN')
('Vinken', 'NOUN')
(',', '.')
('61', 'NUM')
('years', 'NOUN')
('old', 'ADJ')
(',', '.')
('will', 'VERB')
('join', 'VERB')
('the', 'DET')
('board', 'NOUN')
('as', 'ADP')
('a', 'DET')
('nonexecutive', 'ADJ')
('director', 'NOUN')
('Nov.', 'NOUN')
('29', 'NUM')
('.', '.')
('Mr.', 'NOUN')
('Vinken', 'NOUN')
('is', 'VERB')
('chairman', 'NOUN')
('of', 'ADP')
('Elsevier', 'NOUN')
('N.V.', 'NOUN')
(',', '.')
('the', 'DET')
('Dutch', 'NOUN')
('publishing', 'VERB')
('group', 'NOUN')
('.', '.')


In [31]:
# split data into training and validation set in the ratio 80:20
train_set,test_set =train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state = 101)

In [32]:
# create list of train and test tagged words
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
print(len(train_tagged_words))
print(len(test_tagged_words))

80310
20366


In [33]:
# check some of the tagged words.
train_tagged_words[:5]

[('Drink', 'NOUN'),
 ('Carrier', 'NOUN'),
 ('Competes', 'VERB'),
 ('With', 'ADP'),
 ('Cartons', 'NOUN')]

In [34]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)
 
# check total words in vocabulary
vocab = {word for word,tag in train_tagged_words}

12
{'PRON', 'PRT', 'NUM', 'VERB', 'ADJ', 'CONJ', '.', 'NOUN', 'DET', 'X', 'ADV', 'ADP'}


In [35]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
 
     
    return (count_w_given_tag, count_tag)

In [36]:
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [37]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(tags_matrix)

[[6.83371304e-03 1.41230067e-02 6.83371304e-03 4.84738052e-01
  7.06150308e-02 5.01138950e-03 4.19134386e-02 2.12756261e-01
  9.56719834e-03 8.83826911e-02 3.69020514e-02 2.23234631e-02]
 [1.76125243e-02 1.17416831e-03 5.67514673e-02 4.01174158e-01
  8.29745606e-02 2.34833662e-03 4.50097844e-02 2.50489235e-01
  1.01369865e-01 1.21330721e-02 9.39334650e-03 1.95694715e-02]
 [1.42806140e-03 2.60621198e-02 1.84219927e-01 2.07068902e-02
  3.53445187e-02 1.42806144e-02 1.19243130e-01 3.51660132e-01
  3.57015361e-03 2.02427700e-01 3.57015361e-03 3.74866128e-02]
 [3.55432779e-02 3.06629837e-02 2.28360966e-02 1.67955801e-01
  6.63904250e-02 5.43278083e-03 3.48066315e-02 1.10589318e-01
  1.33609578e-01 2.15930015e-01 8.38858187e-02 9.23572779e-02]
 [1.94174761e-04 1.14563107e-02 2.17475723e-02 1.14563107e-02
  6.33009672e-02 1.68932043e-02 6.60194159e-02 6.96893215e-01
  5.24271838e-03 2.09708735e-02 5.24271838e-03 8.05825219e-02]
 [6.03732169e-02 4.39077942e-03 4.06147093e-02 1.50384188e-01
  1

In [38]:
# convert the matrix to a df for better readability
#the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,PRON,PRT,NUM,VERB,ADJ,CONJ,.,NOUN,DET,X,ADV,ADP
PRON,0.006834,0.014123,0.006834,0.484738,0.070615,0.005011,0.041913,0.212756,0.009567,0.088383,0.036902,0.022323
PRT,0.017613,0.001174,0.056751,0.401174,0.082975,0.002348,0.045010,0.250489,0.101370,0.012133,0.009393,0.019569
NUM,0.001428,0.026062,0.184220,0.020707,0.035345,0.014281,0.119243,0.351660,0.003570,0.202428,0.003570,0.037487
VERB,0.035543,0.030663,0.022836,0.167956,0.066390,0.005433,0.034807,0.110589,0.133610,0.215930,0.083886,0.092357
ADJ,0.000194,0.011456,0.021748,0.011456,0.063301,0.016893,0.066019,0.696893,0.005243,0.020971,0.005243,0.080583
CONJ,0.060373,0.004391,0.040615,0.150384,0.113611,0.000549,0.035126,0.349067,0.123491,0.009330,0.057080,0.055982
.,0.068769,0.002789,0.078210,0.089690,0.046132,0.060079,0.092372,0.218539,0.172192,0.025641,0.052569,0.092908
NOUN,0.004659,0.043935,0.009144,0.149134,0.012584,0.042454,0.240094,0.262344,0.013106,0.028825,0.016895,0.176827
DET,0.003306,0.000287,0.022855,0.040247,0.206411,0.000431,0.017393,0.635906,0.006037,0.045134,0.012074,0.009918
X,0.054200,0.185086,0.003075,0.206419,0.017682,0.010379,0.160869,0.061695,0.056890,0.075726,0.025754,0.142226


- Optimization using Viterbi 

In [39]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [40]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)      #define a random seed to get same sentences when run multiple times
 
# choose random 10 numbers
rndom = [random.randint(1,len(test_set)) for x in range(10)]
 
# list of 10 sents on which we test the model
test_run = [test_set[i] for i in rndom]
 
# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]
 
# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [41]:
#Here We will only test 10 sentences to check the accuracy
#as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)


Time taken in seconds:  17.423856258392334
Viterbi Algorithm Accuracy:  93.77990430622009


In [ ]:
#Code to test all the test sentences
#(takes alot of time to run s0 we wont run it here)
# tagging the test sentences()
test_tagged_words = [tup for sent in test_set for tup in sent]
test_untagged_words = [tup[0] for sent in test_set for tup in sent]
test_untagged_words
 
start = time.time()
tagged_seq = Viterbi(test_untagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(test_tagged_words, test_untagged_words) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)


# will take around 30 minutes

In [ ]:
#To improve the performance,we specify a rule base tagger for unknown words 
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                   # nouns
]
 
# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [ ]:
#modified Viterbi to include rule based tagger in it
def Viterbi_rule_based(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
        
         
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
        else:
            if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)]                
             
         
        state.append(state_max)
    return list(zip(words, state))

In [ ]:
# Extract the tags from tagged_seq
predicted_tags = [tag for word, tag in tagged_seq]

# Calculate accuracy by comparing predicted_tags to test_run_base
check = [i for i, j in zip(predicted_tags, test_run_base) if i == j]

accuracy = len(check) / len(predicted_tags)
print('Viterbi Algorithm Accuracy: ', accuracy * 100)


Viterbi Algorithm Accuracy:  0.0


In [ ]:
#Check how a sentence is tagged by the two POS taggers
#and compare them
test_sent="Will can see Marry"
pred_tags_rule=Viterbi_rule_based(test_sent.split())
pred_tags_withoutRules= Viterbi(test_sent.split())
print(pred_tags_rule)
print(pred_tags_withoutRules)
#Will and Marry are tagged as NUM as they are unknown words for Viterbi Algorithm 


[('Will', 'NOUN'), ('can', 'VERB'), ('see', 'VERB'), ('Marry', 'NOUN')]
[('Will', 'PRON'), ('can', 'VERB'), ('see', 'VERB'), ('Marry', 'PRON')]
